In [ ]:
!pip install sqlalchemy
!pip install pymssql

In [ ]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from joblib import dump

In [ ]:
model = RandomForestRegressor(
  random_state=0
)